## Level0 Nodes

In [1]:
import torch
import torchhd

from src.encoding.configs_and_constants import VSAModel
from src.utils.utils import cartesian_bind_tensor, TupleIndexer, flatten_counter
from collections import defaultdict, Counter

vsa = VSAModel.MAP
D = 112 * 112  # 96 didn't give 100%
fruits = torchhd.random(28, D, vsa=vsa.value)
veggies = torch.cat([torchhd.identity(1, D, vsa=vsa.value), torchhd.random(5, D, vsa=vsa.value)], dim=0)
node_codebook = cartesian_bind_tensor([fruits, veggies])
node_indexer = TupleIndexer([len(fruits), len(veggies)])

# Length 33
basket = [
    (0, 1),
    (0, 2),
    (0, 2),
    (0, 2),
    (0, 2),
    (0, 2),
    (0, 3),
    (2, 2),
    (2, 3),
    (0, 3),
    (2, 1),
    (0, 3),
    (0, 2),
    (0, 3),
    (0, 3),
    (0, 1),
    (2, 3),
    (0, 3),
    (0, 2),
    (0, 2),
    (0, 3),
    (0, 1),
    (0, 2),
    (0, 3),
    (0, 1),
    (0, 3),
    (0, 1),
    (0, 2),
    (0, 3),
    (1, 1),
    (2, 2),
    (0, 3),
    (5, 2),
]

print(basket)
print(len(basket))
unique_nodes = set(basket)
print(f"Unique Nodes -> {unique_nodes=}, {len(set(basket))=} ")
nodes_ground_truth = node_indexer.get_idxs(basket)
print(f"{sorted(set(nodes_ground_truth))=}")

nodes = [node_codebook[node_indexer.get_idx(t)] for t in basket]
embedding_0 = torchhd.multiset(torch.stack(nodes, dim=0))

[(0, 1), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 3), (2, 2), (2, 3), (0, 3), (2, 1), (0, 3), (0, 2), (0, 3), (0, 3), (0, 1), (2, 3), (0, 3), (0, 2), (0, 2), (0, 3), (0, 1), (0, 2), (0, 3), (0, 1), (0, 3), (0, 1), (0, 2), (0, 3), (1, 1), (2, 2), (0, 3), (5, 2)]
33
Unique Nodes -> unique_nodes={(0, 1), (2, 1), (1, 1), (0, 3), (2, 3), (0, 2), (2, 2), (5, 2)}, len(set(basket))=8 
sorted(set(nodes_ground_truth))=[1, 2, 3, 7, 13, 14, 15, 32]


### Decoding Level 0 Nodes

In [2]:
from collections import Counter

d = torchhd.dot(embedding_0, node_codebook)
d = d / D
sim_node = torch.round(d).int().clamp(min=0)
## This should print 3, 0, 1, 0, 1, 0, 3, .... so many time as there items.
## And it does, so it should work
print(sim_node)
print(sim_node.sum().item())
decoded_node_idxs = sim_node.nonzero(as_tuple=True)[0]
decoded_node_counts = sim_node[decoded_node_idxs]
# Build the counter
node_counter = Counter(dict(zip(decoded_node_idxs.tolist(), decoded_node_counts.tolist())))
print(node_counter)
nodes_decoded = flatten_counter(node_counter)
print(nodes_decoded)
nodes_basket_dec = list(map(lambda x: node_indexer.get_tuple(x), nodes_decoded))
print(nodes_basket_dec)
print(f"{sorted(nodes_basket_dec) == sorted(basket)=}")

MAPTensor([ 0,  5, 10, 11,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  2,  2,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
          dtype=torch.int32)
33
Counter({3: 11, 2: 10, 1: 5, 14: 2, 15: 2, 7: 1, 13: 1, 32: 1})
[1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 7, 13, 14, 14, 15, 15, 32]
[(0, 1), (0, 1)

## Level 1 - GraphHD Encoding

In [3]:
## Length 72
edge_indexes = [
    (0, 1),
    (1, 0),
    (1, 2),
    (2, 1),
    (2, 3),
    (3, 2),
    (3, 4),
    (4, 3),
    (4, 5),
    (5, 4),
    (5, 6),
    (6, 5),
    (6, 7),
    (6, 32),
    (7, 6),
    (7, 8),
    (8, 7),
    (8, 9),
    (8, 31),
    (9, 8),
    (9, 10),
    (9, 11),
    (10, 9),
    (11, 9),
    (11, 12),
    (11, 28),
    (12, 11),
    (12, 13),
    (13, 12),
    (13, 14),
    (13, 27),
    (14, 13),
    (14, 15),
    (14, 16),
    (15, 14),
    (16, 14),
    (16, 17),
    (16, 25),
    (17, 16),
    (17, 18),
    (17, 23),
    (18, 17),
    (18, 19),
    (19, 18),
    (19, 20),
    (20, 19),
    (20, 21),
    (20, 22),
    (21, 20),
    (22, 20),
    (22, 23),
    (23, 17),
    (23, 22),
    (23, 24),
    (24, 23),
    (25, 16),
    (25, 26),
    (25, 27),
    (26, 25),
    (27, 13),
    (27, 25),
    (28, 11),
    (28, 29),
    (28, 30),
    (29, 28),
    (30, 28),
    (30, 31),
    (31, 8),
    (31, 30),
    (31, 32),
    (32, 6),
    (32, 31),
]

print(f"{len(edge_indexes)=}")

edge_dict = defaultdict(list)
for src_tp, dst_tp in edge_indexes:
    edge_dict[src_tp].append(dst_tp)
print(edge_dict)
print(len(edge_dict))

## Create the memories for each node within the graph, the memory of a node is bundling of hv of all it's neighbours
memories = []
for _, dsts in edge_dict.items():
    memories.append(
        torchhd.multiset(torch.stack([node_codebook[node_indexer.get_idx(basket[d])] for d in dsts], dim=0))
    )

memory_codebook = torch.stack(memories)
assert memory_codebook.shape == (len(basket), D)


src_idxs = torch.tensor([node_indexer.get_idx(basket[s]) for s in edge_dict])

## Create GraphHD like Graph
stacked = torch.stack([node_codebook[src_idxs], memory_codebook], dim=0).transpose(0, 1)
bound = torchhd.multibind(stacked)
graph_hd_level1 = torchhd.multiset(bound) * 0.5

## Easier way to perform the same
graph_hd_level1_hashmap = torchhd.hash_table(node_codebook[src_idxs], memory_codebook) * 0.5
assert torch.equal(graph_hd_level1, graph_hd_level1_hashmap)

len(edge_indexes)=72
defaultdict(<class 'list'>, {0: [1], 1: [0, 2], 2: [1, 3], 3: [2, 4], 4: [3, 5], 5: [4, 6], 6: [5, 7, 32], 7: [6, 8], 8: [7, 9, 31], 9: [8, 10, 11], 10: [9], 11: [9, 12, 28], 12: [11, 13], 13: [12, 14, 27], 14: [13, 15, 16], 15: [14], 16: [14, 17, 25], 17: [16, 18, 23], 18: [17, 19], 19: [18, 20], 20: [19, 21, 22], 21: [20], 22: [20, 23], 23: [17, 22, 24], 24: [23], 25: [16, 26, 27], 26: [25], 27: [13, 25], 28: [11, 29, 30], 29: [28], 30: [28, 31], 31: [8, 30, 32], 32: [6, 31]})
33


## Threshhold Definition

In [4]:
from torch import Tensor
import numpy as np
import torchhd


def determine_threshold(
    node_hvs: Tensor,
    node_memories: Tensor,
    edges: list[tuple[int, int]],
    node_indexer: TupleIndexer,
    graph_nodes: list[tuple, tuple],
):
    """
    Determines the threshold T for edge existence in GrapHD based on the similarity distributions.

    Parameters
    ----------
    node_hvs : [|Total Combinatorial Nodes|, D]]  -> requires node_indexer for indexings
        Dictionary of node identifiers mapped to their hypervectors.
    node_memories : [|Noeds in G|, D]
        Dictionary of node identifiers mapped to their reconstructed node memories.
    edges :
        Set of tuples (src, dst) representing the actual edges in the graph.
    dimension : int
        Dimensionality D of the hypervectors.

    Returns
    -------
    float
        Calculated threshold T for determining edge existence.

    References
    ----------
    The statistical properties referenced from Poduval et al. (2022):
    - For nodes with an actual edge, similarity R follows Gaussian N(1, sqrt((d - 1)/D)).
    - For nodes without an edge, similarity R follows Gaussian N(0, sqrt(d/D)).
    """
    existing_similarities = []
    non_existing_similarities = []

    # Compute similarities for existing edges
    for src, dst in edges:
        dst_idx = node_indexer.get_idx(graph_nodes[dst])
        sim = torchhd.dot(node_memories[src], node_hvs[dst_idx]).item() / D
        existing_similarities.append(sim)

    # Compute similarities for non-existing edges
    for src in range(len(graph_nodes)):
        for dst in range(len(graph_nodes)):
            if src != dst and (src, dst) not in edges:
                dst_idx = node_indexer.get_idx(graph_nodes[dst])
                sim = torchhd.dot(node_memories[src], node_hvs[dst_idx]).item() / D
                non_existing_similarities.append(sim)

    # According to Poduval et al., both signal and noise follow Gaussian distributions:
    # Signal distribution: N(1, sqrt((dA - 1) / D))
    # Noise distribution: N(0, sqrt(dA / D))
    # Hence, the optimal threshold is chosen to maximize separation between these distributions.

    # Empirical mean and standard deviations
    signal_mean, signal_std = np.mean(existing_similarities), np.std(existing_similarities)
    noise_mean, noise_std = np.mean(non_existing_similarities), np.std(non_existing_similarities)

    # Threshold T is selected at intersection of Gaussian curves:
    # Since it's impractical to find analytically, approximate with average of means.
    threshold = (signal_mean + noise_mean) / 2

    # This method matches the description from Poduval et al. (see ROC curve analysis, Figure 4B).

    return threshold

In [5]:
threshhold = determine_threshold(memory_codebook, node_codebook, edge_indexes, node_indexer, basket)
print(threshhold)

0.06388234282534151


## Graph Reconstruction

In [33]:
import torchhd
from torch import Tensor

def reconstruct_graph(
        node_hvs: Tensor,
        graph_memory: Tensor,
        threshold: float,
        nodes_decoded: list[int],
        max_iterations: int = 10,
) -> set[tuple[int, int]]:
    """
    GrapHD graph reconstruction via iterative noise cancellation.
    Based on Poduval et al. (2022), Section 4.3 and Figure 6.

    Parameters:
    ----------
    node_hvs : Tensor [|Nodes|, D]
        Node hypervectors, indexed by node index.
    graph_memory : Tensor [D]
        The graph memory hypervector EG.
    threshold : float
        Similarity threshold T (dot product normalized by D).
    nodes_decoded : list[int]
        List of decoded node indices.
    max_iterations : int
        Maximum number of iterations to refine the graph.

    Returns:
    -------
    edges : set[tuple[int, int]]
        The reconstructed set of edges.
    """

    D = node_hvs.shape[1]
    num_nodes = len(nodes_decoded)

    # Step 1. Initialize f^1(A,B) = 0 for all edges
    edge_predictions = {(src, dst): 0 for src in range(num_nodes) for dst in range(num_nodes) if src != dst}

    # Step 2. First estimation: Check all edges against graph memory
    for (src, dst) in edge_predictions.keys():
        edge_hv = node_hvs[nodes_decoded[src]].bind(node_hvs[nodes_decoded[dst]])
        similarity = torchhd.dot(graph_memory, edge_hv).item() / D
        if similarity > threshold:
            edge_predictions[(src, dst)] = 1

    # Iterate until convergence
    for iteration in range(max_iterations):
        updated_predictions = edge_predictions.copy()

        for (src, dst) in edge_predictions.keys():
            edge_hv = node_hvs[nodes_decoded[src]].bind(node_hvs[nodes_decoded[dst]])
            # Step 3a. Compute EN(k)AB = EG(k) - f(k)(A,B) * EHA*EHB (per paper Section 4.3, eqn)
            if edge_predictions[(src, dst)] == 1:
                noise_memory = graph_memory - edge_hv
            else:
                noise_memory = graph_memory

            # Step 3b. Check similarity: EG(k) - EN(k)AB ?= EHA*EHB
            residual_memory = graph_memory - noise_memory
            similarity = torchhd.dot(residual_memory, edge_hv).item() / D

            # Step 3c. Update f(k+1)(A,B)
            if similarity > threshold:
                updated_predictions[(src, dst)] = 1
            else:
                updated_predictions[(src, dst)] = 0

        # Step 4. Check convergence
        if updated_predictions == edge_predictions:
            print(f"Converged after {iteration+1} iterations")
            break

        edge_predictions = updated_predictions

    # Return edges detected after convergence
    edges = {(src, dst) for (src, dst), exists in edge_predictions.items() if exists}
    return edges

In [37]:
edges = reconstruct_graph(node_codebook, graph_hd_level1, threshhold, nodes_decoded, max_iterations=500)
print(len(edges))
print(f"{threshhold=}")
print(f"Decode Edges (length: {len(edges)}): {sorted(edges)}")
print(f"Real Edges (length: len({len(edge_indexes)}): {sorted(edge_indexes)}")

Converged after 1 iterations
960
threshhold=0.06388234282534151
Decode Edges (length: 960): [(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (0, 24), (0, 25), (0, 27), (0, 30), (0, 31), (1, 0), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (1, 24), (1, 25), (1, 27), (1, 30), (1, 31), (2, 0), (2, 1), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 22), (2, 23), (2, 24), (2, 25), (2, 27), (2, 30), (2, 31), (3, 0), (3, 1), (3, 2), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (3, 11), (3, 12), (3, 13), (3, 14), (3, 15), (3, 16), (3, 17), (3, 18), (3, 19), (3, 20), (3, 21)

In [43]:
import torch
import torchhd
from torchhd import MAPTensor

def reconstruct_graph_2(node_hvs: list[MAPTensor],
                      graph_memory: MAPTensor,
                      threshold: float,
                      max_iterations: int = 15):
    """
    Reconstruct the edge set from a GrapHD graph memory hypervector (EG)
    via iterative noise cancellation, per Section 4.3 of Poduval et al.

    Parameters
    ----------
    node_hvs : list of MAPTensor, length N
        The D-dimensional hypervector for each node EH_i.
    graph_memory : MAPTensor, shape (D,)
        The encoded graph hypervector EG.
    threshold : float
        Decision threshold T. If similarity > T, we predict an edge.
    max_iterations : int
        Maximum noise-cancellation iterations (e.g. 15).

    Returns
    -------
    edges : set of (i, j) tuples
        The reconstructed directed edges.
    """

    N = len(node_hvs)
    D = graph_memory.shape[0]

    # --- Step (•a): Precompute one hypervector per possible edge EH_i * EH_j
    edge_pairs = [(i, j) for i in range(N) for j in range(N) if i != j]
    edge_hvs = [node_hvs[i].bind(node_hvs[j]) for (i, j) in edge_pairs]

    # --- Step (•b): Initial inference f^(1)(i,j) = 1 if δ(EG, EH_i*EH_j) > T
    # δ here = (dot / D)
    f = []
    for eh in edge_hvs:
        sim = (eh.dot(graph_memory) / D).item()
        f.append(1 if sim > threshold else 0)

    # --- Steps (•c) & (•d): Iterative noise cancellation
    for iteration in range(max_iterations):
        # Build EG^(k) = sum over predicted edges of EH_i*EH_j
        EGk = MAPTensor(torch.zeros(D))
        for flag, eh in zip(f, edge_hvs):
            if flag:
                EGk = EGk + eh

        f_next = []
        # For each candidate (i,j):
        for idx, eh in enumerate(edge_hvs):
            # EN^(k)_{ij} = EG^(k) - f^(k)_{ij} * (EH_i*EH_j)
            EN = EGk - (f[idx] * eh)

            # residual = original EG - EN
            residual = graph_memory - EN

            # New decision: δ(residual, EH_i*EH_j)
            sim = (eh.dot(residual) / D).item()
            f_next.append(1 if sim > threshold else 0)

        # Check convergence
        if f_next == f:
            # print(f"Reconstruction converged in {iteration+1} iterations")
            break
        f = f_next

    # Collect all (i,j) with f=1
    edges = {edge_pairs[idx] for idx, flag in enumerate(f) if flag}
    return edges

In [42]:
node_list = node_codebook.tolist()
edges = reconstruct_graph_2(node_list, graph_hd_level1, threshhold)
print(len(edges))
print(f"{threshhold=}")
print(f"Decode Edges (length: {len(edges)}): {sorted(edges)}")
print(f"Real Edges (length: len({len(edge_indexes)}): {sorted(edge_indexes)}")

AttributeError: 'list' object has no attribute 'bind'